# Классификация рыб и котов

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [59]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [60]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [61]:
train_data_path = "dataset/fish_cats/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [62]:
val_data_path = "dataset/fish_cats/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [63]:
test_data_path = "dataset/fish_cats/test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 
batch_size=64

In [64]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

In [65]:
train_features, train_labels = next(iter(train_data_loader))

In [66]:
train_labels, train_features[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([[[-0.6965, -0.6965, -0.6109,  ..., -0.2684, -0.3027, -0.4397],
          [-0.6965, -0.6794, -0.5938,  ..., -0.3027, -0.2856, -0.3883],
          [-0.6281, -0.5938, -0.5767,  ..., -0.2856, -0.3198, -0.3369],
          ...,
          [-0.3198, -0.3198, -0.2513,  ..., -1.1760, -0.8164, -0.4911],
          [-0.3712, -0.3027, -0.2856,  ..., -1.4329, -1.2617, -0.8335],
          [-0.3027, -0.2342, -0.2856,  ..., -1.4500, -1.4500, -1.2617]],
 
         [[-0.6702, -0.7052, -0.6352,  ..., -0.2500, -0.2500, -0.4076],
          [-0.6877, -0.6877, -0.6001,  ..., -0.2325, -0.2150, -0.3550],
          [-0.6176, -0.6001, -0.5826,  ..., -0.1975, -0.2150, -0.2675],
          ...,
          [-0.1099, -0.1099, -0.0224,  ..., -1.2654, -0.9678, -0.8277],
          [-0.1625, -0.074

# Our First Model, SimpleNet

In [67]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x     

In [68]:
simplenet = SimpleNet()

## Create an optimizer

In [69]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [70]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## Training

In [71]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device=device):
    for epoch in range(epochs):
        training_loss = 0
        valid_loss = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))
            

In [72]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

C:\Users\Matebook 14S\AppData\Local\Temp\ipykernel_16068\897253786.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)


Epoch: 0, Training Loss: 2.63, Validation Loss: 4.74, accuracy = 0.28
Epoch: 1, Training Loss: 2.38, Validation Loss: 1.58, accuracy = 0.52
Epoch: 2, Training Loss: 1.26, Validation Loss: 0.54, accuracy = 0.77
Epoch: 3, Training Loss: 0.49, Validation Loss: 0.72, accuracy = 0.70
Epoch: 4, Training Loss: 0.44, Validation Loss: 0.72, accuracy = 0.73


In [73]:
labels = ['cat','fish']
img = Image.open(FILENAME)
img = transforms(img)
img = img.unsqueeze(0)
prediction = simplenet(img)
prediction = prediction.argmax()
print(labels[prediction])

NameError: name 'FILENAME' is not defined

In [ ]:
torch.save(simplenet, "simplenet")

In [ ]:
simplenet = torch.load("simplenet")